# cointegration

Use the "Run" button to execute the code.

In [36]:
jovian.commit(project="cointegration")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "kaustavsen16/cointegration" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/kaustavsen16/cointegration


'https://jovian.ai/kaustavsen16/cointegration'

In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas import DataFrame
import pathlib
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from statsmodels.tsa.api import VAR, VECM
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.vector_ar.vecm import coint_johansen,select_coint_rank

In [3]:
datainddf = pd.read_csv('DATASHEET.csv')
datainddf

,YEAR,LAB_SHARE,inv_share,log_gdp_capita
0,1980,38.500,3.025565,9.949954
1,1981,37.800,2.910870,9.985044
2,1982,38.500,2.906384,9.995924
3,1983,38.400,2.901579,10.043075
4,1984,39.600,2.913217,10.057612
5,1985,39.700,2.938682,10.086223
6,1986,40.600,2.927267,10.110673
7,1987,41.300,3.012120,10.127722
8,1988,39.800,3.036231,10.198175
9,1989,39.900,3.056615,10.234838


In [4]:
datainddf.index = pd.date_range(start ="1979",end ="2018",freq='A')

In [5]:
datainddf=datainddf.drop(["YEAR"], axis=1)

In [6]:
def joh_output(res):
    output = pd.DataFrame([res.lr2,res.lr1],
                          index=['max_eig_stat',"trace_stat"])
    print(output.T,'\n')
    print("Critical values(90%, 95%, 99%) of max_eig_stat\n",res.cvm,'\n')
    print("Critical values(90%, 95%, 99%) of trace_stat\n",res.cvt,'\n')
 

In [7]:
# Model 3 (2 lag-difference used = 3 lags VAR or VAR(3) model)
# with constant/trend (deterministc) term
joh_model3 = coint_johansen(datainddf,0,4) # k_ar_diff +1 = K
joh_output(joh_model3)
# Model 2: with linear trend only
joh_model2 = coint_johansen(datainddf,1,4) # k_ar_diff +1 = K
joh_output(joh_model2)
 
# Model 1: no constant/trend (deterministc) term
joh_model1 = coint_johansen(datainddf,-1,4) # k_ar_diff +1 = K
joh_output(joh_model1)

   max_eig_stat  trace_stat
0     24.069991   48.377552
1     15.866700   24.307562
2      8.440861    8.440861 

Critical values(90%, 95%, 99%) of max_eig_stat
 [[18.8928 21.1314 25.865 ]
 [12.2971 14.2639 18.52  ]
 [ 2.7055  3.8415  6.6349]] 

Critical values(90%, 95%, 99%) of trace_stat
 [[27.0669 29.7961 35.4628]
 [13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]] 

   max_eig_stat  trace_stat
0     20.706477   33.507928
1      9.123880   12.801451
2      3.677570    3.677570 

Critical values(90%, 95%, 99%) of max_eig_stat
 [[21.8731 24.2522 29.2631]
 [15.0006 17.1481 21.7465]
 [ 2.7055  3.8415  6.6349]] 

Critical values(90%, 95%, 99%) of trace_stat
 [[32.0645 35.0116 41.0815]
 [16.1619 18.3985 23.1485]
 [ 2.7055  3.8415  6.6349]] 

   max_eig_stat  trace_stat
0     22.516951   33.394271
1     10.185107   10.877320
2      0.692212    0.692212 

Critical values(90%, 95%, 99%) of max_eig_stat
 [[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]] 

Cr

In [34]:
rank = select_coint_rank(datainddf,0,3, method='maxeig', signif=0.01)
rank.summary()

r_0,r_1,test statistic,critical value
0,1,32.38,25.86
1,2,18.85,18.52
2,3,5.453,6.635


In [19]:
datainddf['D_inv_share']=datainddf['inv_share'].diff(1)

In [29]:
adftest = adfuller(datainddf['D_inv_share'].dropna(),autolag="BIC")
print('ADF Statistic: %f' % adftest[0])
print('p-value: %f' % adftest[1])

ADF Statistic: -7.347382
p-value: 0.000000


In [32]:
model = VECM(datainddf, coint_rank=2,k_ar_diff=3)
vecm_res = model.fit()
vecm_res.gamma.round(4)
vecm_res.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
L1.LAB_SHARE,0.0436,0.195,0.224,0.823,-0.338,0.425
L1.inv_share,-4.7178,4.326,-1.091,0.275,-13.196,3.760
L1.log_gdp_capita,11.1975,14.383,0.779,0.436,-16.992,39.387
L2.LAB_SHARE,0.0632,0.199,0.318,0.750,-0.326,0.452
L2.inv_share,-1.4408,4.457,-0.323,0.746,-10.176,7.294
L2.log_gdp_capita,22.7865,14.001,1.628,0.104,-4.655,50.228
L3.LAB_SHARE,-0.0765,0.185,-0.413,0.680,-0.440,0.287
L3.inv_share,-2.2584,3.668,-0.616,0.538,-9.448,4.932
L3.log_gdp_capita,15.5831,13.450,1.159,0.247,-10.779,41.945
,coef,std err,z,P>|z|,[0.025,0.975]
